# Lesson 36: Python Advanced - Context manager

## Context manager - intro

In [1]:
# Context manager is important when we open and close a file, init and then stop some program or create and close.
# It appears with a special command, which is recommended:

with open(r"mydata.txt", "w+") as file:
    file.writelines("Success")
    
# here file.close() is called automatically

In [3]:
# we change the name of the file and create a new file:

with open(r"mydata**.txt", "w+") as file:
    file.writelines("Success")

In [8]:
# Now we create our own context manager. Let us build a class:

import time

class time_measure:
    
    def __init__(self):
        pass
    
    # To create a manager we need to have the method __enter__ and __exit__
    
    def __enter__(self):
        print("entering ... ")
        self.__start = time.time()
        return self
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        print("exiting ... ")
        self.__stop = time.time()
        self.__difference = self.__stop - self.__start
        print("Execution time: {}".format(self.__difference))
        
# Now I use this class. Note to add () for the class. And in this case we do not have any name for the file so 
# we can ignore "as..."

with time_measure():
    time.sleep(3)

entering ... 
exiting ... 
Execution time: 3.000458240509033


## Context manager - file ini

In [21]:
# We will create a new content manager (class) to handle a file:

import os

class ini_file:
    def __init__(self, path):
        self.path = path
        # Here we will be storing data read from disk:
        self.parameters = {}
        self.read_from_disk()
        
    # We need this method:
    
    def read_from_disk(self):
        # checking if the file exists:
        if os.path.isfile(self.path):    
            with open(self.path) as file:
                for line in file:
                    parts = line.replace("\n", "").split("=")
                    self.parameters[parts[0]] = parts[1]
    
    # Checking if a parameter exists:
    
    def read_parameter(self, key):
        if key in self.parameters.keys():
            return self.parameters[key]
        else:
            return None
          
    # Writing the parameter:
    
    def write_parameter(self, key, value):
        self.parameters[key] = value
        
    def save_on_disk(self):
        with open(self.path, "w") as file:
            for key, value in self.parameters.items():
                line = "{}={}\n".format(key, value)
                file.writelines(line)
                
    def __enter__(self):
        return self
    
    def __exit__(self, exc_type, exc_value, exc_tb):
        pass
    
    
# Creating a file a writing parameters there:

ini = ini_file(r"file.ini")
ini.write_parameter("version", 1)
ini.write_parameter("level", "advanced")
ini.save_on_disk()

In [23]:
# Reading the parameters off the existing file:

ini_2 = ini_file(r"file.ini")
print(ini_2.parameters)
print(ini_2.read_parameter("version"))

{'version': '1', 'level': 'advanced'}
1


In [25]:
# Or using context manager:

with ini_file(r"file.ini") as ini_3:
    print(ini_3.parameters)
    print(ini_3.read_parameter("level"))

{'version': '1', 'level': 'advanced'}
advanced


## Context manager - error handling

In [35]:
# We will be working with the same class which we just made: 

# In case of the class of context manager I can handle error through the function __exit__

import os

class ini_file:
    def __init__(self, path):
        self.path = path
        # Here we will be storing data read from disk:
        self.parameters = {}
        self.read_from_disk()
        
    # We need this method:
    
    def read_from_disk(self):
        # checking if the file exists:
        if os.path.isfile(self.path):    
            with open(self.path) as file:
                for line in file:
                    parts = line.replace("\n", "").split("=")
                    self.parameters[parts[0]] = parts[1]
    
    # Checking if a parameter exists:
    
    def read_parameter(self, key):
        if key in self.parameters.keys():
            return self.parameters[key]
        else:
            return None
          
    # Writing the parameter:
    
    def write_parameter(self, key, value):
        self.parameters[key] = value
        
    def save_on_disk(self):
        with open(self.path, "w") as file:
            for key, value in self.parameters.items():
                line = "{}={}\n".format(key, value)
                file.writelines(line)
                
    def __enter__(self):
        return self
    
    def __exit__(self, exc_type, exc_value, exc_tb):
        print("exc_type = {}".format(exc_type))
        print("exc_value = {}".format(exc_value))
        print("exc_tb = {}".format(exc_tb))
     
        # We can decide whether we want to see the error of it should be hidden.
        if exc_type == OSError:
            return False
        else:
            return True
    
with ini_file(r"file**.ini") as myini:
    myini.write_parameter("mode", "strict")
    myini.write_parameter("loglevel", "light")
    myini.save_on_disk()
    # I have an error here, and immediately __exit__ handles it. Because it is not OSError, __exit__ returns 
    # True and thus the description of the error is hidden. But the info about the error is given by exc_type, etc
    print(10/0)

exc_type = <class 'ZeroDivisionError'>
exc_value = division by zero
exc_tb = <traceback object at 0x7fcb05cc2bc0>


## Contextlib

In [45]:
# Here we will build a class of context manager, which does not use __enter__ and __exit__, but it will be 
# using decorators:

class Door:
    
    def __init__(self, where):
        self.where = where
    
    def open(self):
        print("Opening door to the {}".format(self.where))
        
    def close(self):
        print("Closing door to the {}".format(self.where))
        
door1 = Door("hell")
door2 = Door("future")

door1.open()
door1.close()

# But note that this class will not work for "with class() as sth:" statement, there will be an error.

# To avoid it I can define a fuction and indicate via decorator that this function should use context manager:

from contextlib import contextmanager

@contextmanager
def OpenAndClose(obj):
    obj.open()
    yield obj
    obj.close()

with OpenAndClose(Door("next room")) as door:
    print("This is entrance to {}".format(door.where))

    
print("______")

# The same thing can be done using (because most people forget to close things)

@contextmanager
def OnlyClose(obj):
    yield obj
    obj.close()

with OnlyClose(Door("next room")) as door:
    door.open()
    print("This is entrance to {}".format(door.where))

Opening door to the hell
Closing door to the hell
Opening door to the next room
This is entrance to next room
Closing door to the next room
______
Opening door to the next room
This is entrance to next room
Closing door to the next room


In [47]:
from urllib.request import urlopen
from contextlib import closing

# "closing" is a special object which is built to close things
# "urlopen" opens and load a website

with closing(urlopen("http://www.kursyonline24.eu")) as page:
    for line in page:
        print(line)

b'<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Strict//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-strict.dtd">\r\n'
b'<html xmlns="http://www.w3.org/1999/xhtml" lang="pl-PL">\r\n'
b'<head>\r\n'
b'\t<meta http-equiv="Content-Type" content="text/html; charset=utf-8" />\r\n'
b'\t<meta name="viewport" content="width=device-width, initial-scale=1.0" />   \r\n'
b'    <meta name="google-site-verification" content="2BOHI7hfRMXvKNTBrw1mrFukrrq4Z34dsxl6wU4H-i0" /> \r\n'
b'\t<title>Kursy Online | Kompletne tutoriale po polsku, bo grunt to dobrze zrozumie\xc4\x87</title>\r\n'
b'\t<link rel="profile" href="http://gmpg.org/xfn/11" />\r\n'
b'\t\t<title>Kursy Online &#8211; Kompletne tutoriale po polsku, bo grunt to dobrze zrozumie\xc4\x87</title>\n'
b"<link rel='dns-prefetch' href='//fonts.googleapis.com' />\n"
b"<link rel='dns-prefetch' href='//s.w.org' />\n"
b'<link rel="alternate" type="application/rss+xml" title="Kursy Online &raquo; Kana\xc5\x82 z wpisami" href="https://www.kursyonline24.eu/fee

b'\t\t\t\t\t\t<div class="slide-text-bg2"><h3>Ju\xc5\xbc 2500 student\xc3\xb3w pozna\xc5\x82o podstawy pracy z Linuxem i jest gotowych do pracy z tym systemem na powa\xc5\xbcnie. Te\xc5\xbc do\xc5\x82\xc4\x85cz!</h3></div><div class="flex-btn-div"><a href="https://www.kursyonline24.eu/linux/" class="btn1 flex-btn">Czytaj dalej</a></div>\t\t\t\t\t\n'
b'                    </div>\n'
b'\t\t\t</li>\t\n'
b'\t\t\t\t\t\t\t<li>\n'
b'\t\t\t\t\t<img width="1460" height="685" src="https://www.kursyonline24.eu/wp-content/uploads/2017/08/Splash_screen_sql4.png" class="img-responsive wp-post-image" alt="" srcset="https://www.kursyonline24.eu/wp-content/uploads/2017/08/Splash_screen_sql4.png 1460w, https://www.kursyonline24.eu/wp-content/uploads/2017/08/Splash_screen_sql4-300x141.png 300w, https://www.kursyonline24.eu/wp-content/uploads/2017/08/Splash_screen_sql4-768x360.png 768w, https://www.kursyonline24.eu/wp-content/uploads/2017/08/Splash_screen_sql4-1024x480.png 1024w" sizes="(max-width: 1460px)

In [54]:
# Removing a file:

import os

# os.remove("file**.ini")

# But if I try to remove it again there will be an error, because this file does not exist now.
# To handle it, I can use "suppress" from cotextlib, to hide this error, because it is not harmful:

from contextlib import suppress

with suppress(FileNotFoundError):
    os.remove("file*.ini")


In [56]:
# Next nice function is redirect_stdout() - it prints everything into a given file and not to this notebook:

from contextlib import redirect_stdout

f = open(r"red_file.txt", "w")

with redirect_stdout(f):
    print("Hello")
    d = Door("exit")
    d.open()
    d.close()